In [ ]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://app.yamol.tw/exam/24548"
resonpse = requests.get(url)
soup = BeautifulSoup(resonpse.text,"html.parser")
print(soup.prettify()) # soup.prettify() 會將 html 格式化

In [ ]:
# 靜態爬蟲無法爬蟲以下網頁的問題內容，因此使用動態爬蟲。
question_div = soup.select_one('div[class*="MuiBox-root"]')

if question_div:
    question_content = question_div.get_text(strip=True)
    print(question_content)
else:
    print("指定的 div 標籤未找到。")

In [ ]:
# from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import time
import pandas as pd
import numpy as np

# 登入帳號密碼
username = 'your_username'
password = 'your_password'

# 設置 Chrome 瀏覽器選項
chrome_options = Options()
chrome_options.add_argument("--headless")  # 啟用無頭模式
chrome_options.add_argument("--disable-gpu")

# 設置 ChromeDriver 的路徑
driver_path = 'chromedriver.exe'  # 替換為你的 chromedriver 路徑
service = Service(driver_path)

# 初始化 WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# 載入網頁
url = 'https://app.yamol.tw/exam/24548'  # 替換為你的目標 URL
driver.get(url)

# 等待網頁加載完成
time.sleep(5)  # 視情況調整等待時間

question_content = [] # 建立空的 list 儲存題目內容
options_content = [] # 建立空的 list 儲存選項內容
answer_content = [] # 建立空的 list 儲存答案內容


visited_questions = []  # 建立空的 list 儲存已經抓取過的題目
# visited_questions = set() # 用來儲存已經抓取過的題目
  

# 點擊Index切換網頁內容
i = 1
while True:
# for i in range(0,2):
    # visited_questions.append(seiunm)  # 將當前題目添加到已訪問集合中
    # print("visited_questions",visited_questions)
    
    try :
        button = driver.find_element(By.XPATH, f"//button[contains(@class, 'text-info') and text()='{i}. ']")
        print(f"點擊 {i}. 題")
        i = i + 1
        button.click()

    except :
        print("已完成所有頁面")
        

    # 用By.XPATH 定位到 QUESTION 1 並抓取內容
    stat_div = driver.find_element(By.XPATH, "//div[contains(text(), '統計：')]")
    statistics_text = stat_div.text
    print("統計內容：", statistics_text)

    # re 正規表示式 #(\d+)是一個捕獲整數的模式 
    match = re.search(r'#(\d+)', statistics_text)
    if match:
        seiunm = match.group(1)
        new_url = f"{url}?info=item.{seiunm}" # 取得新的URL
        print("新的 URL:", new_url) 

        # if seiunm in visited_questions :
        #     print("已經抓取過所有題目")
        #     break
    
        # 點擊另一個題目頁面
        driver.get(new_url)
        time.sleep(2)
  
# 定位到 題目欄位 並抓取內容
        question_div = driver.find_element(By.CLASS_NAME, 'item-htmlRender')
        inner_divs = question_div.find_element(By.CLASS_NAME, 'MuiBox-root')
        question_content.append(inner_divs.text)
          
# 定位到 選項欄位 並抓取內容
        options_div = driver.find_element(By.ID, f'item-{seiunm}-options')
        inner_divs = options_div.find_elements(By.CLASS_NAME, 'MuiBox-root')
        for div in inner_divs :
            options_content.append(div.text)
            
# 定位到 答案欄位 並抓取內容
        answer_div = driver.find_element(By.XPATH, "//div[contains(text(), '答案：')]/a")
        answer_letter = answer_div.text
        answer_content.append(answer_letter)

        print(f"問題:{question_content}")
        print(f"選項:{options_content}")
        print(f"答案:{answer_letter}")  

        # df = pd.DataFrame(columns=['題目'])
        # df.loc[len(df)] = [question_content]
        # df.to_csv('topic.csv', index=False)
        # 將題目、選項、答案加入 DataFrame
        # df = pd.DataFrame(columns=['題目','A', 'B', 'C', 'D','E','答案','題目編號'])  # 建立空的 DataFrame 儲存題目及答案
        
        while len(options_content)< 5 :
            options_content.append("")
            
        new_row = pd.DataFrame([{
                    '題目': question_content,
                    'A': options_content[0],
                    'B': options_content[1],
                    'C': options_content[2],
                    'D': options_content[3],
                    'E': options_content[4],
                    '答案': answer_content ,
                    '題目代碼': new_url
                }])
        
        df = pd.concat([df, new_row], ignore_index=True)
        
            # df.loc[len(df)] = [question_content, options_content[0], options_content[1], options_content[2], options_content[3],options_content[4], answer_letter,new_url]
        df.to_csv('topic.csv', index=False)
        
# # 關閉瀏覽器
# driver.quit()
        print(df)


In [ ]:
answer_div = driver.find_element(By.XPATH, "//div[contains(text(), '答案：')]/a")
answer_letter = answer_div.text
answer_content.append(answer_div)
print(answer_letter)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time

# 配置 Chrome 驅動
chrome_options = Options()
chrome_options.add_argument("--headless")  # 隱藏瀏覽器
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# 這裡需要指定您的 ChromeDriver 的路徑
driver_path = 'chromedriver.exe'  # 替換為你的 chromedriver 路徑
service = Service(driver_path)

# 初始化 WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# 打開登入頁面

login_url = "https://app.yamol.tw/exam/24548"  # 修改為實際的登入 URL
driver.get(login_url)


# 增加額外的等待時間確保頁面加載完成
time.sleep(5)

# 填寫電子郵件和密碼並登錄
wait = WebDriverWait(driver, 20)  # 增加超時時間至 20 秒

# 填寫電子郵件
wait = WebDriverWait(driver, 10)
email_input = wait.until(EC.presence_of_element_located((By.NAME, 'email')))
email_input.send_keys("email")  # 用您的電子郵件替換

# 填寫密碼
password_input = driver.find_element(By.XPATH, "//input[@name='password']")
password_input.send_keys("password")  # 用您的密碼替換

# 點擊登入按鈕
login_button = driver.find_element(By.XPATH, "//button[@type='submit' and text()='登入']")
login_button.click()

# 等待頁面加載，這裡可以根據需要調整等待時間
time.sleep(5)

In [ ]:
df = df({
            '题目': [question_content],
            'A': [options_content[0]],
            'B': [options_content[1]],
            'C': [options_content[2]],
            'D': [options_content[3]],
            'E': [options_content[4]],
            '答案': [answer_content],
            '新网址': [new_url]
        }, ignore_index=True)